In [22]:
import os 
import MySQLdb
import pandas as pd 
import sqlalchemy

## Criando conexão

In [24]:
#SQL Connection 
user = 'root' 
password = '32313877tv' 
host = 'localhost' 
database = 'survey'
string_conection = f'mysql://{user}:{password}@{host}/{database}'

In [25]:
engine = sqlalchemy.create_engine(string_conection)
conn = engine.connect()

## Lendo o database

![image info](./sql/model.png)

In [11]:
database = "./base"
temp_list = []
for arquivo in os.listdir(database):
    file_path = os.path.join(database, arquivo)
    temp_list.append(pd.read_csv(file_path, sep='|', encoding='latin-1'))

df = pd.concat(temp_list)

In [28]:
df.head(10)

,cod_pessoa,data_coleta,genero,data_nascimento,animal_estimacao,clima,bebida_favorita,hobbies
0,10142398,2020-06-17,Masculino,1998-12-01,gato,moderado,Cerveja,Escutar música
1,10142896,2020-06-17,Masculino,1966-03-10,gato,moderado,Cerveja,Assistir TV
2,10143412,2020-06-17,Masculino,1988-08-30,gato,quente,Água,Escrever
3,10143428,2020-06-17,Masculino,1995-01-07,cachorro,quente,Vinho,Ler livros
0,10141979,2020-07-18,Feminino,1977-03-13,gato,quente,refrigerante,Dormir
1,10142000,2020-07-18,Feminino,1980-06-28,tartaruga,frio,Vinho,Escrever
2,10142017,2020-07-18,Masculino,1966-04-28,cachorro,frio,Água,Escutar música
3,10142276,2020-07-18,Masculino,1955-10-28,tartaruga,frio,Cerveja,Escrever
4,10142413,2020-07-18,Masculino,1980-08-01,cachorro,frio,Cerveja,Assistir TV
5,10142476,2020-07-18,Masculino,1974-11-15,gato,frio,Cerveja,Escrever


## Unificando atributos

In [35]:
df_pessoa = df[['cod_pessoa', 'data_nascimento', 'genero']].drop_duplicates()
df_pessoa

,cod_pessoa,data_nascimento,genero
0,10142398,1998-12-01,Masculino
1,10142896,1966-03-10,Masculino
2,10143412,1988-08-30,Masculino
3,10143428,1995-01-07,Masculino
0,10141979,1977-03-13,Feminino
...,...,...,...
1,10142449,1969-12-09,Masculino
2,10142735,1978-11-15,Masculino
3,10143016,1988-12-20,Masculino
4,10143094,1980-05-22,Masculino


In [55]:
climas = df['clima'].unique()
animal_estimacao = df['animal_estimacao'].unique()
hobbies = df['hobbies'].unique()
bebida_favorita = df['bebida_favorita'].unique()

## Inserindo

In [42]:
for row in df_pessoa.itertuples():
    try:
        #print(row)'
        insert_query = f"""insert into survey.pessoa(cod_pessoa, genero, data_nascimento) values ('{row.cod_pessoa}', '{row.genero}', '{row.data_nascimento}')"""
        conn.execute(insert_query)
    except Exception as e:
        print(f"A error occurs at insertion on table pessoa {e}")

In [57]:
for clima in climas:
    try:
        insert_sql = f"""insert into clima(clima) values ('{clima}')"""
        conn.execute(insert_sql)
        print(f'{clima} foi inserido')
    except Exception as e:
        print(f"A error occurs at insertion on table clima {e}")

moderado foi inserido
quente foi inserido
frio foi inserido


In [51]:
for pet in animal_estimacao:
    try:
        insert_sql = f"""insert into animal_estimacao(animal) values ('{pet}')"""
        conn.execute(insert_sql)
        print(f'{pet} foi inserido')
    except Exception as e:
        print(f"A error occurs at insertion on table clima {e}")

In [54]:
for bebida in bebida_favorita:
    try:
        insert_sql = f"""insert into bebida(bebida) values ('{bebida}')"""
        conn.execute(insert_sql)
        print(f'{bebida} foi inserido')
    except Exception as e:
        print(f"A error occurs at insertion on table clima {e}")

In [56]:
for hobbie in hobbies:
    try:
        insert_sql = f"""insert into hobbie(hobbie) values ('{hobbie}')"""
        conn.execute(insert_sql)
        print(f'{hobbie} foi inserido')
    except Exception as e:
        print(f"A error occurs at insertion on table clima {e}")

## Criando tabela auxiliar para inserção de pesquisas

In [59]:
#Expota-se os dados crus para que posteriormente o relacionamento seja feito
df.to_sql('sgt_pesquisa', con=conn, schema='survey')

In [ ]:
query = """INSERT INTO pesquisa(data_pesquisa, cod_pessoa, cod_animal_estimacao, 
					cod_bebida, cod_clima, cod_hobbie)
			(
				SELECT sgt.data_coleta as data_pesquisa, 
		                sgt.cod_pessoa, 
		                aes.cod_animal_estimacao,
		                beb.cod_bebida,
                        cli.cod_clima, 
                        hob.cod_hobbie 
                FROM sgt_pesquisa as sgt
                JOIN animal_estimacao as aes
                ON sgt.animal_estimacao = aes.animal 
                JOIN bebida as beb
                ON sgt.bebida_favorita = beb.bebida
                JOIN clima as cli
                ON sgt.clima = cli.clima
                JOIN hobbie as hob
                ON sgt.hobbies = hob.hobbie
            ); """
con.execute(query)